In [4]:
import pandas as pd
import numpy as np
selloutData = pd.read_csv("../Python_ST_EX/dataset/kopo_channel_seasonality_new.csv")
selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)
selloutData["YEAR"] = selloutData["YEARWEEK"].str[0:4]
selloutData["WEEK"] = selloutData["YEARWEEK"].str[4:]
selloutData["QTY_NEW"] = np.where(selloutData["QTY"] < 0, 0, selloutData["QTY"])
selloutData[(selloutData.QTY < 0)].head()
refinedSelloutData = selloutData[selloutData["WEEK"].astype(int) <= 52]
refinedSelloutData.tail()

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK,QTY_NEW
124653,A10,PRODUCT60,201630,824.0,2016,30,824.0
124654,A10,PRODUCT56,201630,275.0,2016,30,275.0
124655,A10,PRODUCT61,201630,0.0,2016,30,0.0
124656,A10,PRODUCT12,201630,15021.0,2016,30,15021.0
124657,A10,PRODUCT1,201630,568.0,2016,30,568.0


In [2]:
def project_seasonality_index(dataFrame, groupKey, aggColumn, movingAverage = [15, 9, 9], sortKey = None, ascending = True):   
    import math
    import numpy as np

    import timeit
    start = timeit.default_timer()
    # Your code start    
    
    if (sortKey != None):
        sortedDataFrame = dataFrame.sort_values(sortKey, ascending = ascending, inplace = False)
    else:
        sortedDataFrame = dataFrame
        
    groupedDataFrame = sortedDataFrame.reset_index(drop = True)
    
    rowLength = len(groupedDataFrame)
    
    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'DataSort')
    
    import timeit
    start = timeit.default_timer()
    # Your code start
    
    groupListForMatch = [""] * rowLength
    for i in groupKey:
        groupListForMatch += groupedDataFrame[i].astype(str)
    sameGroup = []
    for i in range(0, rowLength):
        if (i == 0):
            matchGroup1 = groupListForMatch[i]
            sameGroup = [i]
        else:
            matchGroup2 = groupListForMatch[i]
        if (i > 0):
            if (matchGroup1 != matchGroup2):
                matchGroup1 = matchGroup2
                sameGroup += [i]
    
    sameGroupLength = len(sameGroup)
    
    updownRange = math.floor(movingAverage[0] / 2)

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'Grouping')

    start = timeit.default_timer()
    
    aggColumnList = list(groupedDataFrame[aggColumn])
    
    movingAverageList = []
    for i in range(0, sameGroupLength):
        if (i < sameGroupLength - 1):
            for j in range(sameGroup[i], sameGroup[i + 1]):
                perMovingAverage = [aggColumnList[j]]
                for k in range(1, updownRange + 1):
                    if (j - k >= sameGroup[i]):
                        perMovingAverage += [aggColumnList[j - k]]
                    if (j + k <= sameGroup[i + 1] - 1):
                        perMovingAverage += [aggColumnList[j + k]]
                movingAverageList += [np.mean(perMovingAverage)]
        else:
            for j in range(sameGroup[i], rowLength):
                perMovingAverage = [aggColumnList[j]]
                for k in range(1, updownRange + 1):
                    if (j - k >= sameGroup[i]):
                        perMovingAverage += [aggColumnList[j - k]]
                    if (j + k <= rowLength - 1):
                        perMovingAverage += [aggColumnList[j + k]]
                movingAverageList += [np.mean(perMovingAverage)]
    
    columnName = 'MA'
    columnNameMatch = columnName
    tempCount = 1
    while(list(groupedDataFrame.columns).count(columnNameMatch) > 0):
        tempCount += 1
        columnNameMatch = columnName + str(tempCount)
    columnName = columnNameMatch
    groupedDataFrame[columnName] = movingAverageList

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'MovingAverage')
    
    start = timeit.default_timer()
    
    updownRange = math.floor(movingAverage[1] / 2)

    movingStdList = []
    for i in range(0, sameGroupLength):
        if (i < sameGroupLength - 1):
            for j in range(sameGroup[i], sameGroup[i + 1]):
                perMovingStd = [movingAverageList[j]]
                for k in range(1, updownRange + 1):
                    if (j - k >= sameGroup[i]):
                        perMovingStd += [movingAverageList[j - k]]
                    if (j + k <= sameGroup[i + 1] - 1):
                        perMovingStd += [movingAverageList[j + k]]
                if (sameGroup[i + 1] - sameGroup[i] > 1):
                    movingStdList += [np.std(perMovingStd, ddof = 1)]
                else:
                    movingStdList += [math.inf]
        else:
            for j in range(sameGroup[i], rowLength):
                perMovingStd = [movingAverageList[j]]
                for k in range(1, updownRange + 1):
                    if (j - k >= sameGroup[i]):
                        perMovingStd += [movingAverageList[j - k]]
                    if (j + k <= rowLength - 1):
                        perMovingStd += [movingAverageList[j + k]]
                if (rowLength - sameGroup[i] > 1):
                    movingStdList += [np.std(perMovingStd, ddof = 1)]
                else:
                    movingStdList += [math.inf]
    
    columnName = 'MSTD'
    columnNameMatch = columnName
    tempCount = 1
    while(list(groupedDataFrame.columns).count(columnNameMatch) > 0):
        tempCount += 1
        columnNameMatch = columnName + str(tempCount)
    columnName = columnNameMatch
    groupedDataFrame[columnName] = movingStdList

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'MovingStandardDeviation')

    start = timeit.default_timer()
    
    refinedDataList = []
    for i in range(0, rowLength):
        if (aggColumnList[i] >= movingAverageList[i] + movingStdList[i]):
            refinedDataList += [movingAverageList[i] + movingStdList[i]]
        elif (aggColumnList[i] <= movingAverageList[i] - movingStdList[i]):
            refinedDataList += [movingAverageList[i] - movingStdList[i]]
        else:
            refinedDataList += [aggColumnList[i]]

    columnName = 'REFINED_' + aggColumn
    columnNameMatch = columnName
    tempCount = 1
    while(list(groupedDataFrame.columns).count(columnNameMatch) > 0):
        tempCount += 1
        columnNameMatch = columnName + str(tempCount)
    columnName = columnNameMatch
    groupedDataFrame[columnName] = refinedDataList
    
    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'DataRefine')
    
    start = timeit.default_timer()
    
    updownRange = math.floor(movingAverage[2] / 2)
    
    trendlineList = []
    for i in range(0, sameGroupLength):
        if (i < sameGroupLength - 1):
            for j in range(sameGroup[i], sameGroup[i + 1]):
                perTrendline = [refinedDataList[j]]
                for k in range(1, updownRange + 1):
                    if (j - k >= sameGroup[i]):
                        perTrendline += [refinedDataList[j - k]]
                    if (j + k <= sameGroup[i + 1] - 1):
                        perTrendline += [refinedDataList[j + k]]
                trendlineList += [np.mean(perTrendline)]
        else:
            for j in range(sameGroup[i], rowLength):
                perTrendline = [refinedDataList[j]]
                for k in range(1, updownRange + 1):
                    if (j - k >= sameGroup[i]):
                        perTrendline += [refinedDataList[j - k]]
                    if (j + k <= rowLength - 1):
                        perTrendline += [refinedDataList[j + k]]
                trendlineList += [np.mean(perTrendline)]
    
    columnName = 'TRENDLINE'
    columnNameMatch = columnName
    tempCount = 1
    while(list(groupedDataFrame.columns).count(columnNameMatch) > 0):
        tempCount += 1
        columnNameMatch = columnName + str(tempCount)
    columnName = columnNameMatch
    groupedDataFrame[columnName] = trendlineList

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'Trendline')
    
    start = timeit.default_timer()
    
    seasonalityIndexList = []
    for i in range(0, rowLength):
        if (aggColumnList[i] > 0):
            seasonalityIndexList += [aggColumnList[i] / trendlineList[i]]
        else:
            seasonalityIndexList += [1]

    columnName = 'SEASONALITY_INDEX'
    columnNameMatch = columnName
    tempCount = 1
    while(list(groupedDataFrame.columns).count(columnNameMatch) > 0):
        tempCount += 1
        columnNameMatch = columnName + str(tempCount)
    columnName = columnNameMatch
    groupedDataFrame[columnName] = seasonalityIndexList
    
    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'SeasonalityIndex')
    
    return groupedDataFrame

# issues
# 1. 값이 0일 경우
# 3. 코드 실행 시간
# 4. 변수명 수정 groupedDataFrame

In [5]:
import timeit
start = timeit.default_timer()
# Your code start

sortKey = ['REGIONID','PRODUCT','YEARWEEK']
a = project_seasonality_index(refinedSelloutData, groupKey=['REGIONID','PRODUCT',"YEAR"], aggColumn='QTY_NEW', sortKey = sortKey)

stop = timeit.default_timer()
print('Time: ' , stop - start, 'TotalTime')

Time:  0.08678906599999436 DataSort
Time:  1.436169011000004 Grouping
Time:  1.7858093609999912 MovingAverage
Time:  3.888448955000001 MovingStandardDeviation
Time:  0.1672161600000095 DataRefine
Time:  1.660964891000006 Trendline
Time:  0.06862687899999287 SeasonalityIndex
Time:  9.185069853000002 TotalTime


In [6]:
# import math
# a[(a.MSTD != math.inf)]
a

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK,QTY_NEW,MA,MSTD,REFINED_QTY_NEW,TRENDLINE,SEASONALITY_INDEX
0,A00,PRODUCT34,201401,661.0,2014,01,661.0,520.000000,12.002994,532.002994,528.452136,1.250823
1,A00,PRODUCT34,201402,679.0,2014,02,679.0,514.444444,15.933923,530.378367,513.970894,1.321086
2,A00,PRODUCT34,201403,578.0,2014,03,578.0,516.800000,19.930825,536.730825,500.692490,1.154401
3,A00,PRODUCT34,201404,532.0,2014,04,532.0,503.363636,23.784855,527.148492,488.500596,1.089047
4,A00,PRODUCT34,201405,516.0,2014,05,516.0,490.750000,30.428905,516.000000,486.444974,1.060757
5,A00,PRODUCT34,201406,423.0,2014,06,423.0,480.230769,38.666084,441.564685,479.122985,0.882863
6,A00,PRODUCT34,201407,407.0,2014,07,407.0,468.214286,47.192218,421.022068,461.192056,0.882496
7,A00,PRODUCT34,201408,364.0,2014,08,364.0,456.666667,53.509329,403.157338,440.666408,0.826022
8,A00,PRODUCT34,201409,470.0,2014,09,470.0,431.533333,59.591171,470.000000,421.427687,1.115257
9,A00,PRODUCT34,201410,538.0,2014,10,538.0,402.066667,64.038426,466.105093,398.761020,1.349179


In [13]:
a.to_csv("a2.csv", index = False)

In [112]:
def dataManipulation2(dataFrame, movingAverage, groupKey, aggColumn, aggFunc = 'mean', sortKey = None, ascending = True):
    
    import math
    import numpy as np
    import pandas as pd

    import timeit
    start = timeit.default_timer()
    # Your code start    

    if (sortKey != None):
        sortedDataFrame = dataFrame.sort_values(sortKey, ascending = ascending, inplace = False)
    else:
        sortedDataFrame = dataFrame

#     if (groupKey != None):
#         if (aggColumn != None):
#             groupedDataFrame = sortedDataFrame.groupby(groupKey).agg(aggFunc)[aggColumn].reset_index()
#         else:
#             groupedDataFrame = sortedDataFrame.groupby(groupKey).agg(aggFunc).reset_index()
#     else:
#         groupedDataFrame = sortedDataFrame
    groupedDataFrame = sortedDataFrame
    
    rowLength = len(groupedDataFrame)
    
    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'DataSort')
    
    import timeit
    start = timeit.default_timer()
    # Your code start
    
    matchGroup1 = []
    sameGroup = []
    for i in range(0, rowLength):
        matchGroup2 = []
        for j in groupKey:
            if (i == 0):
                matchGroup1 += [groupedDataFrame[j][i]]
                sameGroup = [i]
            else:
                matchGroup2 += [groupedDataFrame[j][i]]
        matchBoolean = matchGroup1 == matchGroup2
        if (i > 0):
            if (matchBoolean == False):
                matchGroup1 = matchGroup2
                sameGroup += [i]
    
    sameGroupLength = len(sameGroup)
    
    updownRange = math.floor(15 / 2)

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'Grouping')

    start = timeit.default_timer()
    
    aggColumnList = list(groupedDataFrame[aggColumn])
    
    def movingAverageCalculation():
        movingAverageList = []
        for i in range(0, sameGroupLength):
            if (i < sameGroupLength - 1):
                for j in range(sameGroup[i], sameGroup[i + 1]):
                    perMovingAverage = [aggColumnList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingAverage += [aggColumnList[j - k]]
                        if (j + k < sameGroup[i + 1] - 1):
                            perMovingAverage += [aggColumnList[j + k]]
                    movingAverageList += [np.mean(perMovingAverage)]
            else:
                for j in range(sameGroup[i], rowLength):
                    perMovingAverage = [aggColumnList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingAverage += [aggColumnList[j - k]]
                        if (j + k < rowLength - 1):
                            perMovingAverage += [aggColumnList[j + k]]
                    movingAverageList += [np.mean(perMovingAverage)]
        return movingAverageList
    
    movingAverageTotalList = movingAverageCalculation()
    groupedDataFrame['MA'] = movingAverageTotalList

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'MovingAverage')
    
    start = timeit.default_timer()
    
    updownRange = math.floor(9 / 2)

    def movingStdCalculation():
        movingStdList = []
        for i in range(0, sameGroupLength):
            if (i < sameGroupLength - 1):
                for j in range(sameGroup[i], sameGroup[i + 1]):
                    perMovingStd = [movingAverageTotalList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingStd += [movingAverageTotalList[j - k]]
                        if (j + k < sameGroup[i + 1] - 1):
                            perMovingStd += [movingAverageTotalList[j + k]]
                    if (sameGroup[i + 1] - sameGroup[i] > 1):
                        movingStdList += [np.std(perMovingStd, ddof = 1)]
                    else:
                        movingStdList += [np.nan]
            else:
                for j in range(sameGroup[i], rowLength):
                    perMovingStd = [movingAverageTotalList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingStd += [movingAverageTotalList[j - k]]
                        if (j + k < rowLength - 1):
                            perMovingStd += [movingAverageTotalList[j + k]]
                    if (rowLength - sameGroup[i] > 1):
                        movingStdList += [np.std(perMovingStd, ddof = 1)]
                    else:
                        movingStdList += [np.nan]
        return movingStdList
    
    movingStdTotalList = movingStdCalculation()
    groupedDataFrame['MSTD'] = movingStdTotalList

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'StandardDeviation')

    start = timeit.default_timer()
    
    refinedDataList = []
    for i in range(0, rowLength):
        if (groupedDataFrame[aggColumn][i] > movingAverageTotalList[i] + movingStdTotalList[i]):
            refinedDataList += [movingAverageTotalList[i] + movingStdTotalList[i]]
        elif (groupedDataFrame[aggColumn][i] < movingAverageTotalList[i] - movingStdTotalList[i]):
            refinedDataList += [movingAverageTotalList[i] - movingStdTotalList[i]]
        else:
            refinedDataList += [groupedDataFrame[aggColumn][i]]

    groupedDataFrame['REFINED_' + aggColumn] = refinedDataList
    
    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'DataRefine')
    
    start = timeit.default_timer()
    
    updownRange = math.floor(9 / 2)
    
    def trendlineCalculation():
        trendlineList = []
        for i in range(0, sameGroupLength):
            if (i < sameGroupLength - 1):
                for j in range(sameGroup[i], sameGroup[i + 1]):
                    perTrendline = [refinedDataList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perTrendline += [refinedDataList[j - k]]
                        if (j + k < sameGroup[i + 1] - 1):
                            perTrendline += [refinedDataList[j + k]]
                    trendlineList += [np.mean(perTrendline)]
            else:
                for j in range(sameGroup[i], rowLength):
                    perTrendline = [refinedDataList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perTrendline += [refinedDataList[j - k]]
                        if (j + k < rowLength - 1):
                            perTrendline += [refinedDataList[j + k]]
                    trendlineList += [np.mean(perTrendline)]
        return trendlineList
    
    trendlineTotalList = trendlineCalculation()
    groupedDataFrame['TRENDLINE'] = trendlineTotalList

    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'Trendline')
    
    start = timeit.default_timer()
    
    seasonalityIndexList = []
    for i in range(0, rowLength):
        if (trendlineTotalList[i] > 0):
            seasonalityIndexList += [groupedDataFrame[aggColumn][i] / trendlineTotalList[i]]
        else:
            seasonalityIndexList += [np.nan]

    groupedDataFrame['SEASONALITY_INDEX'] = seasonalityIndexList
    
    stop = timeit.default_timer()
    print('Time: ' , stop - start, 'SeasonalityIndex')
    
    return groupedDataFrame

In [7]:
def dataManipulation2(dataFrame, movingAverage, groupKey, aggColumn, aggFunc = 'mean', sortKey = None, ascending = True):
    import math
    import numpy as np
    import pandas as pd
    
    if (sortKey != None):
        sortedDataFrame = dataFrame.sort_values(sortKey, ascending = ascending, inplace = False)
    else:
        sortedDataFrame = dataFrame

#     if (groupKey != None):
#         if (aggColumn != None):
#             groupedDataFrame = sortedDataFrame.groupby(groupKey).agg(aggFunc)[aggColumn].reset_index()
#         else:
#             groupedDataFrame = sortedDataFrame.groupby(groupKey).agg(aggFunc).reset_index()
#     else:
#         groupedDataFrame = sortedDataFrame
    groupedDataFrame = sortedDataFrame
    
    rowLength = len(groupedDataFrame)
    
    matchGroup1 = []
    sameGroup = []
    for i in range(0, rowLength):
        matchGroup2 = []
        for j in groupKey:
            if (i == 0):
                matchGroup1 += [groupedDataFrame[j][i]]
                sameGroup = [i]
            else:
                matchGroup2 += [groupedDataFrame[j][i]]
        matchBoolean = matchGroup1 == matchGroup2
        if (i > 0):
            if (matchBoolean == False):
                matchGroup1 = matchGroup2
                sameGroup += [i]
    
    sameGroupLength = len(sameGroup)
    
    updownRange = math.floor(15 / 2)

    def movingAverageCalculation():
        movingAverageList = []
        for i in range(0, sameGroupLength):
            if (i < sameGroupLength - 1):
                for j in range(sameGroup[i], sameGroup[i + 1]):
                    perMovingAverage = [groupedDataFrame[aggColumn][j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingAverage += [groupedDataFrame[aggColumn][j - k]]
                        if (j + k < sameGroup[i + 1] - 1):
                            perMovingAverage += [groupedDataFrame[aggColumn][j + k]]
                    movingAverageList += [np.mean(perMovingAverage)]
            else:
                for j in range(sameGroup[i], rowLength):
                    perMovingAverage = [groupedDataFrame[aggColumn][j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingAverage += [groupedDataFrame[aggColumn][j - k]]
                        if (j + k < rowLength - 1):
                            perMovingAverage += [groupedDataFrame[aggColumn][j + k]]
                    movingAverageList += [np.mean(perMovingAverage)]
        return movingAverageList

    movingAverageTotalList = movingAverageCalculation()
    groupedDataFrame['MA'] = movingAverageTotalList

    updownRange = math.floor(9 / 2)

    def movingStdCalculation():
        movingStdList = []
        for i in range(0, sameGroupLength):
            if (i < sameGroupLength - 1):
                for j in range(sameGroup[i], sameGroup[i + 1]):
                    perMovingStd = [movingAverageTotalList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingStd += [movingAverageTotalList[j - k]]
                        if (j + k < sameGroup[i + 1] - 1):
                            perMovingStd += [movingAverageTotalList[j + k]]
                    if (sameGroup[i + 1] - sameGroup[i] > 1):
                        movingStdList += [np.std(perMovingStd, ddof = 1)]
                    else:
                        movingStdList += [np.nan]
            else:
                for j in range(sameGroup[i], rowLength):
                    perMovingStd = [movingAverageTotalList[j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perMovingStd += [movingAverageTotalList[j - k]]
                        if (j + k < rowLength - 1):
                            perMovingStd += [movingAverageTotalList[j + k]]
                    if (rowLength - sameGroup[i] > 1):
                        movingStdList += [np.std(perMovingStd, ddof = 1)]
                    else:
                        movingStdList += [np.nan]
        return movingStdList
    
    movingStdTotalList = movingStdCalculation()
    groupedDataFrame['MSTD'] = movingStdTotalList
    
    refinedDataList = []
    for i in range(0, rowLength):
        if (groupedDataFrame[aggColumn][i] > movingAverageTotalList[i] + movingStdTotalList[i]):
            refinedDataList += [movingAverageTotalList[i] + movingStdTotalList[i]]
        elif (groupedDataFrame[aggColumn][i] < movingAverageTotalList[i] - movingStdTotalList[i]):
            refinedDataList += [movingAverageTotalList[i] - movingStdTotalList[i]]
        else:
            refinedDataList += [groupedDataFrame[aggColumn][i]]

    groupedDataFrame['REFINED_' + aggColumn] = refinedDataList
    
    updownRange = math.floor(9 / 2)
    
    def trendlineCalculation():
        trendlineList = []
        for i in range(0, sameGroupLength):
            if (i < sameGroupLength - 1):
                for j in range(sameGroup[i], sameGroup[i + 1]):
                    perTrendline = [groupedDataFrame['REFINED_' + aggColumn][j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perTrendline += [groupedDataFrame['REFINED_' + aggColumn][j - k]]
                        if (j + k < sameGroup[i + 1] - 1):
                            perTrendline += [groupedDataFrame['REFINED_' + aggColumn][j + k]]
                    trendlineList += [np.mean(perTrendline)]
            else:
                for j in range(sameGroup[i], rowLength):
                    perTrendline = [groupedDataFrame['REFINED_' + aggColumn][j]]
                    for k in range(1, updownRange + 1):
                        if (j - k > sameGroup[i]):
                            perTrendline += [groupedDataFrame['REFINED_' + aggColumn][j - k]]
                        if (j + k < rowLength - 1):
                            perTrendline += [groupedDataFrame['REFINED_' + aggColumn][j + k]]
                    trendlineList += [np.mean(perTrendline)]
        return trendlineList
    
    trendlineTotalList = trendlineCalculation()
    groupedDataFrame['TRENDLINE'] = trendlineTotalList

    seasonalityIndexList = []
    for i in range(0, rowLength):
        seasonalityIndexList += [groupedDataFrame[aggColumn][i] / trendlineTotalList[i]]
    
    groupedDataFrame['SEASONALITY_INDEX'] = seasonalityIndexList
    
    return groupedDataFrame

In [91]:
import timeit
start = timeit.default_timer()
# Your code start

a = project_seasonality_index(sortedData, groupKey=['REGIONID','PRODUCT',"YEAR"], aggColumn='QTY')

stop = timeit.default_timer()
print('Time: ' , stop - start, 'TotalTime')

Time:  0.05531128399979934 DataSort
Time:  1.0895946679993358 Grouping
Time:  1.8483016509999288 MovingAverage
Time:  3.8858787449999 MovingStandardDeviation
Time:  0.12194704200010165 DataRefine
Time:  1.7071671039993817 Trendline
Time:  0.11119351299930713 SeasonalityIndex
Time:  8.865802712000004 TotalTime


In [92]:
a

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK,MA,MSTD,REFINED_QTY,TRENDLINE,SEASONALITY_INDEX
0,A00,PRODUCT34,201401,661.0,2014,01,520.000000,16.575679,536.575679,515.392458,1.282518
1,A00,PRODUCT34,201402,679.0,2014,02,496.125000,14.726203,510.851203,494.591606,1.372850
2,A00,PRODUCT34,201403,578.0,2014,03,500.777778,18.437888,519.215666,480.982393,1.201707
3,A00,PRODUCT34,201404,532.0,2014,04,487.600000,22.040791,509.640791,468.780735,1.134859
4,A00,PRODUCT34,201405,516.0,2014,05,475.272727,25.406223,500.678950,468.933143,1.100370
5,A00,PRODUCT34,201406,423.0,2014,06,465.166667,32.595246,432.571421,467.972082,0.903900
6,A00,PRODUCT34,201407,407.0,2014,07,453.384615,40.448288,412.936327,452.210837,0.900023
7,A00,PRODUCT34,201408,364.0,2014,08,442.071429,46.500641,395.570787,433.631318,0.839423
8,A00,PRODUCT34,201409,470.0,2014,09,431.533333,52.943473,470.000000,416.337897,1.128891
9,A00,PRODUCT34,201410,538.0,2014,10,402.066667,58.216924,460.283590,395.373570,1.360738


In [44]:
sortedData["MA"] = sortedData["QTY"].rolling(window = 15, center = True).mean()
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,2014,01,NaN
1,A00,PRODUCT34,201402,679.0,2014,02,NaN
2,A00,PRODUCT34,201403,578.0,2014,03,NaN
3,A00,PRODUCT34,201404,532.0,2014,04,NaN
4,A00,PRODUCT34,201405,516.0,2014,05,NaN
5,A00,PRODUCT34,201406,423.0,2014,06,NaN
6,A00,PRODUCT34,201407,407.0,2014,07,NaN
7,A00,PRODUCT34,201408,364.0,2014,08,456.666667
8,A00,PRODUCT34,201409,470.0,2014,09,431.533333
9,A00,PRODUCT34,201410,538.0,2014,10,402.066667


In [202]:
a.to_csv("a1.csv",index=False)

In [318]:
def dataManipulation2(dataFrame, movingAverage, groupKey, aggColumn, aggFunc = 'mean', sortKey = None, ascending = True):
    import math
    import numpy as np
    import pandas as pd
    
    if (sortKey != None):
        sortedDataFrame = dataFrame.sort_values(sortKey, ascending = ascending, inplace = False)
    else:
        sortedDataFrame = dataFrame
        
#     if (groupKey != None):
#         if (aggColumn != None):
#             groupedDataFrame = sortedDataFrame.groupby(groupKey).agg(aggFunc)[aggColumn].reset_index()
#         else:
#             groupedDataFrame = sortedDataFrame.groupby(groupKey).agg(aggFunc).reset_index()
#     else:
#         groupedDataFrame = sortedDataFrame
    groupedDataFrame = sortedDataFrame
        
    def movingAverageCalculation(sameGroup):
        updownRange = math.floor(movingAverage / 2)
        movingAverageList = []
        for i in range(sameGroup[0], sameGroup[len(sameGroup)-1] + 1):
            perMovingAverage = [groupedDataFrame[aggColumn][i]]
            for j in range(1, updownRange + 1):
                if (i - j >= sameGroup[0]):
                    perMovingAverage += [groupedDataFrame[aggColumn][i - j]]
                if (i + j <= sameGroup[len(sameGroup)-1]):
                    perMovingAverage += [groupedDataFrame[aggColumn][i + j]]
            movingAverageList += [np.mean(perMovingAverage)]
        return movingAverageList
    
    movingAverageTotalList = []
    matchGroup1 = []
    sameGroup = []
    groupingBoolean = False
    for i in range(0, len(groupedDataFrame)):
        matchGroup2 = []
        for j in groupKey:
            if (i == 0):
                matchGroup1 += [groupedDataFrame[j][i]]
                sameGroup = [i]
            else:
                matchGroup2 += [groupedDataFrame[j][i]]
        matchBoolean = matchGroup1 == matchGroup2
        if (i > 0):
            if (matchBoolean):
                groupingBoolean = True
            else:
                matchGroup1 = matchGroup2
                groupingBoolean = False
        if (i > 0 and i < len(groupedDataFrame) - 1):
            if (groupingBoolean):
                sameGroup += [i]
            else:
                movingAverageTotalList += movingAverageCalculation(sameGroup)
                sameGroup = [i]
        elif (i == len(groupedDataFrame) - 1):
            if (groupingBoolean):
                sameGroup += [i]
                movingAverageTotalList += movingAverageCalculation(sameGroup)
            else:
                movingAverageTotalList += movingAverageCalculation(sameGroup)
                movingAverageTotalList += [groupedDataFrame[aggColumn][i]]
    
    def movingStdCalculation(sameGroup):
        updownRange = math.floor(9 / 2)
        movingStdList = []
        for i in range(sameGroup[0], sameGroup[len(sameGroup)-1] + 1):
            perMovingStd = [movingAverageTotalList[i]]
            for j in range(1, updownRange + 1):
                if (i - j >= sameGroup[0]):
                    perMovingStd += [movingAverageTotalList[i - j]]
                if (i + j <= sameGroup[len(sameGroup)-1]):
                    perMovingStd += [movingAverageTotalList[i + j]]
            if (len(sameGroup) > 1):
                movingStdList += [np.std(perMovingStd, ddof = 1)]
            else:
                movingStdList += [np.nan]
        return movingStdList

    movingStdTotalList = []
    matchGroup1 = []
    sameGroup = []
    groupingBoolean = False
    for i in range(0, len(groupedDataFrame)):
        matchGroup2 = []
        for j in groupKey:
            if (i == 0):
                matchGroup1 += [groupedDataFrame[j][i]]
                sameGroup = [i]
            else:
                matchGroup2 += [groupedDataFrame[j][i]]
        matchBoolean = matchGroup1 == matchGroup2
        if (i > 0):
            if (matchBoolean):
                groupingBoolean = True
            else:
                matchGroup1 = matchGroup2
                groupingBoolean = False
        if (i > 0 and i < len(groupedDataFrame) - 1):
            if (groupingBoolean):
                sameGroup += [i]
            else:
                movingStdTotalList += movingStdCalculation(sameGroup)
                sameGroup = [i]
        elif (i == len(groupedDataFrame) - 1):
            if (groupingBoolean):
                sameGroup += [i]
                movingStdTotalList += movingStdCalculation(sameGroup)
            else:
                movingStdTotalList += movingStdCalculation(sameGroup)
                movingSTdTotalList += [np.nan]
    
    groupedDataFrame['MA'] = movingAverageTotalList
    groupedDataFrame['MSTD'] = movingStdTotalList
    result = groupedDataFrame
    return result

In [26]:
import pandas as pd
selloutData = pd.read_csv("../Python_ST_EX/dataset/kopo_channel_seasonality_new.csv")
selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)
selloutData["YEAR"] = selloutData["YEARWEEK"].str[0:4]
selloutData["WEEK"] = selloutData["YEARWEEK"].str[4:]
refinedSelloutData = selloutData[selloutData["WEEK"].astype(int) <= 52]
refinedSelloutData.tail()

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
124653,A10,PRODUCT60,201630,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,2016,30
124657,A10,PRODUCT1,201630,568.0,2016,30


In [30]:
sortKey = ['REGIONID','PRODUCT','YEARWEEK']
sortedData = selloutData.sort_values(sortKey, inplace = False)
sortedData = sortedData.reset_index()
sortedData = sortedData[['REGIONID', 'PRODUCT', 'YEARWEEK', 'QTY', 'YEAR', 'WEEK']]
sortedData.tail()

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
124653,A77,PRODUCT12,201648,4152.0,2016,48
124654,A77,PRODUCT12,201649,5086.0,2016,49
124655,A77,PRODUCT12,201650,5846.0,2016,50
124656,A77,PRODUCT12,201651,4933.0,2016,51
124657,A77,PRODUCT12,201652,7551.0,2016,52


In [333]:
groupKey = ['REGIONID', 'PRODUCT', 'YEAR']
groupData = sortedData.groupby(groupKey).agg(['mean'])['QTY']
groupData = groupData.reset_index()
groupData.columns = ['REGIONID', 'PRODUCT', 'YEAR','QTY']
groupData.head()

,REGIONID,PRODUCT,YEAR,QTY
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,85.698113
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,7.547170


In [21]:
import numpy as np

In [22]:
np.std([520.000000,496.125000,500.777778,487.600000,475.272727],ddof=1)

16.57567912660601

In [338]:
groupedDataFrame = sortedData
movingStdTotalList = []
matchGroup1 = []
sameGroup = []
for i in range(0, len(groupedDataFrame)):
    matchGroup2 = []
    for j in groupKey:
        if (i == 0):
            matchGroup1 += [groupedDataFrame[j][i]]
            sameGroup = [i]
        else:
            matchGroup2 += [groupedDataFrame[j][i]]
    matchBoolean = matchGroup1 == matchGroup2
    if (i > 0):
        if (matchBoolean == False):
            sameGroup += [i]
            matchGroup1 = matchGroup2

print(sameGroup)

[0, 52, 105, 157, 209, 262, 314, 366, 419, 471, 523, 576, 628, 680, 733, 785, 837, 890, 942, 994, 1047, 1099, 1151, 1204, 1256, 1308, 1361, 1413, 1465, 1518, 1570, 1622, 1675, 1727, 1779, 1832, 1884, 1936, 1989, 2041, 2093, 2146, 2198, 2250, 2303, 2355, 2407, 2460, 2512, 2564, 2617, 2669, 2721, 2774, 2826, 2878, 2931, 2983, 3035, 3088, 3140, 3192, 3245, 3297, 3349, 3402, 3454, 3506, 3559, 3611, 3663, 3716, 3768, 3820, 3873, 3925, 3977, 4030, 4082, 4134, 4187, 4239, 4291, 4344, 4396, 4448, 4501, 4553, 4605, 4658, 4710, 4762, 4815, 4867, 4919, 4972, 5024, 5076, 5129, 5181, 5233, 5286, 5338, 5390, 5443, 5495, 5547, 5600, 5652, 5704, 5757, 5809, 5861, 5914, 5966, 6018, 6071, 6123, 6175, 6228, 6280, 6332, 6385, 6437, 6489, 6542, 6594, 6646, 6699, 6751, 6803, 6856, 6908, 6960, 7013, 7065, 7117, 7170, 7222, 7274, 7327, 7379, 7431, 7484, 7536, 7588, 7641, 7693, 7745, 7798, 7850, 7902, 7955, 8007, 8059, 8112, 8164, 8216, 8269, 8321, 8373, 8426, 8478, 8530, 8583, 8635, 8687, 8740, 8792, 8844, 88

In [367]:
groupedDataFrame

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,2014,01
1,A00,PRODUCT34,201402,679.0,2014,02
2,A00,PRODUCT34,201403,578.0,2014,03
3,A00,PRODUCT34,201404,532.0,2014,04
4,A00,PRODUCT34,201405,516.0,2014,05
5,A00,PRODUCT34,201406,423.0,2014,06
6,A00,PRODUCT34,201407,407.0,2014,07
7,A00,PRODUCT34,201408,364.0,2014,08
8,A00,PRODUCT34,201409,470.0,2014,09
9,A00,PRODUCT34,201410,538.0,2014,10


In [366]:
movingAverageList = []
movingAverage = 3
aggColumn = 'QTY'
updownRange = movingAverage
for i in range(0, len(sameGroup)):
    if (i < len(sameGroup) - 1):
        for j in range(sameGroup[i], sameGroup[i + 1]):
            perMovingAverage = [groupedDataFrame[aggColumn][j]]
            for k in range(1, updownRange + 1):
                if (j - k > sameGroup[i]):
                    perMovingAverage += [groupedDataFrame[aggColumn][j - k]]
                if (j + k < sameGroup[i + 1] - 1):
                    perMovingAverage += [groupedDataFrame[aggColumn][j + k]]
            movingAverageList += [np.mean(perMovingAverage)]
    else:
        for j in range(sameGroup[i], len(groupedDataFrame)):
            perMovingAverage = [groupedDataFrame[aggColumn][j]]
            for k in range(1, updownRange + 1):
                if (j - k > sameGroup[i]):
                    perMovingAverage += [groupedDataFrame[aggColumn][j - k]]
                if (j + k < len(groupedDataFrame) - 1):
                    perMovingAverage += [groupedDataFrame[aggColumn][j + k]]
            movingAverageList += [np.mean(perMovingAverage)]
movingAverageList

[612.5,
 593.2,
 564.8333333333334,
 542.2857142857143,
 499.85714285714283,
 470.0,
 464.2857142857143,
 441.0,
 417.57142857142856,
 407.7142857142857,
 394.14285714285717,
 384.2857142857143,
 357.7142857142857,
 314.7142857142857,
 287.7142857142857,
 260.42857142857144,
 229.85714285714286,
 201.85714285714286,
 173.71428571428572,
 145.0,
 121.28571428571429,
 104.85714285714286,
 90.42857142857143,
 82.14285714285714,
 87.14285714285714,
 112.28571428571429,
 176.42857142857142,
 230.42857142857142,
 283.0,
 326.42857142857144,
 364.14285714285717,
 387.14285714285717,
 382.85714285714283,
 339.2857142857143,
 302.42857142857144,
 267.85714285714283,
 242.14285714285714,
 213.42857142857142,
 197.14285714285714,
 190.14285714285714,
 178.0,
 173.0,
 174.14285714285714,
 183.57142857142858,
 186.0,
 174.71428571428572,
 177.57142857142858,
 174.71428571428572,
 169.85714285714286,
 165.0,
 148.0,
 144.5,
 150.75,
 134.4,
 123.0,
 114.0,
 99.85714285714286,
 78.57142857142857,
 71

In [187]:
rowLength = len(sortedData)
groupKey = ['REGIONID','PRODUCT','YEAR']
groupedDataFrame = sortedData
groupListForMatch = [""] * rowLength

for i in groupKey:
    groupListForMatch += groupedDataFrame[i].astype(str)

sameGroup = []
matchGroup1 = ""
for i in range(0, rowLength):
    matchGroup2 = ""
    if (i == 0):
        matchGroup1 = groupListForMatch[i]
        sameGroup = [i]
    else:
        matchGroup2 = groupListForMatch[i]
    matchGroupBoolean = matchGroup1 == matchGroup2
    if (i > 0):
        if (matchGroupBoolean == True):
            matchGroup1 = matchGroup2
            sameGroup += [i]
print([groupListForMatch])

[0         A00PRODUCT342014
1         A00PRODUCT342014
2         A00PRODUCT342014
3         A00PRODUCT342014
4         A00PRODUCT342014
5         A00PRODUCT342014
6         A00PRODUCT342014
7         A00PRODUCT342014
8         A00PRODUCT342014
9         A00PRODUCT342014
10        A00PRODUCT342014
11        A00PRODUCT342014
12        A00PRODUCT342014
13        A00PRODUCT342014
14        A00PRODUCT342014
15        A00PRODUCT342014
16        A00PRODUCT342014
17        A00PRODUCT342014
18        A00PRODUCT342014
19        A00PRODUCT342014
20        A00PRODUCT342014
21        A00PRODUCT342014
22        A00PRODUCT342014
23        A00PRODUCT342014
24        A00PRODUCT342014
25        A00PRODUCT342014
26        A00PRODUCT342014
27        A00PRODUCT342014
28        A00PRODUCT342014
29        A00PRODUCT342014
                ...       
124628    A77PRODUCT122016
124629    A77PRODUCT122016
124630    A77PRODUCT122016
124631    A77PRODUCT122016
124632    A77PRODUCT122016
124633    A77PRODUCT122016
